In [1]:
#importing dependencies
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from bestbuy import *
from microcenter import *
from fuzzywuzzy import fuzz
from sqlalchemy import create_engine

C:\Users\nhle9\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [81]:
#creating the main df that we will be adding onto
gpu_csv = pd.read_csv('data/gpu_specs_prices - gpu_specs_prices.csv')
gpu_csv.head()

,name,brand,model,memory,memory_interface,length,interface,chipset,base_clock,clock_speed,frame_sync,price,used
0,MSI Gaming GeForce RTX 3090 24GB GDRR6X 384-Bi...,MSI,RTX 3090 GAMING X TRIO 24G,24 GB,GDRR6X,335 mm,PCIe x16,GeForce RTX 3090,1395 MHz,1785 MHz,G-Sync,"$1,289.99",0
1,"EVGA GeForce RTX 3090 FTW3 Ultra Gaming, 24GB ...",EVGA,RTX 3090 FTW3 Ultra,24 GB,GDDR6X,300 mm,PCIe x16,GeForce RTX 3090,1395 MHz,1800 MHz,G-Sync,"$1,199.99",0
2,ASUS ROG Strix NVIDIA GeForce RTX 3090 White O...,ASUS,ROG Strix RTX 3090 OC Edition,24 GB,GDDR6X,318.5 mm,PCIe x16,GeForce RTX 3090,1395 MHz,1890 MHz,G-Sync,"$1,794.99",0
3,MSI Gaming GeForce RTX 3090 24GB GDRR6X 384-Bi...,MSI,RTX 3090 SUPRIM X 24G,24 GB,GDDR6X,336 mm,PCIe x16,GeForce RTX 3090,1395 MHz,1875 MHz,G-Sync,"$2,017.21",0
4,GIGABYTE AORUS GeForce RTX 3090 Xtreme 24G Gra...,Gigabyte,AORUS RTX 3090 Xtreme 24G,24 GB,GDDR6X,319 mm,PCIe x16,GeForce RTX 3090,1395 MHz,1860 MHz,G-Sync,"$3,399.99",0


In [82]:
#grabbing the column names to drop what we dont need
gpu_csv.columns

Index(['name', 'brand', 'model', 'memory', 'memory_interface', 'length',
       'interface', 'chipset', 'base_clock', 'clock_speed', 'frame_sync',
       'price', 'used'],
      dtype='object')

In [83]:
#dropping unnecessary columns and changing price column name to be amazon
gpu_csv.rename(columns={'price': 'Amazon Price'}, inplace=True)
gpu_csv.drop('used',axis=1,inplace=True)

In [84]:
#creating keywords to match the models with the names from other retailers that dont have the same model#
model_keywords = []

In [85]:
for index, row in gpu_csv.iterrows():
    #splitting the string in the model column
    keywords = row['model'].lower().split()
    #inputting the brand as some have the same model name but different brand
    keywords.insert(0, row['brand'].lower())
    model_keywords.append(keywords)

In [86]:
#model_keywords
gpu_csv.insert(3,'keywords',model_keywords)

In [87]:
#double check
gpu_csv.rename(columns = {'Amazon Price':'amazon'}, inplace=True)
gpu_csv

,name,brand,model,keywords,memory,memory_interface,length,interface,chipset,base_clock,clock_speed,frame_sync,amazon
0,MSI Gaming GeForce RTX 3090 24GB GDRR6X 384-Bi...,MSI,RTX 3090 GAMING X TRIO 24G,"[msi, rtx, 3090, gaming, x, trio, 24g]",24 GB,GDRR6X,335 mm,PCIe x16,GeForce RTX 3090,1395 MHz,1785 MHz,G-Sync,"$1,289.99"
1,"EVGA GeForce RTX 3090 FTW3 Ultra Gaming, 24GB ...",EVGA,RTX 3090 FTW3 Ultra,"[evga, rtx, 3090, ftw3, ultra]",24 GB,GDDR6X,300 mm,PCIe x16,GeForce RTX 3090,1395 MHz,1800 MHz,G-Sync,"$1,199.99"
2,ASUS ROG Strix NVIDIA GeForce RTX 3090 White O...,ASUS,ROG Strix RTX 3090 OC Edition,"[asus, rog, strix, rtx, 3090, oc, edition]",24 GB,GDDR6X,318.5 mm,PCIe x16,GeForce RTX 3090,1395 MHz,1890 MHz,G-Sync,"$1,794.99"
3,MSI Gaming GeForce RTX 3090 24GB GDRR6X 384-Bi...,MSI,RTX 3090 SUPRIM X 24G,"[msi, rtx, 3090, suprim, x, 24g]",24 GB,GDDR6X,336 mm,PCIe x16,GeForce RTX 3090,1395 MHz,1875 MHz,G-Sync,"$2,017.21"
4,GIGABYTE AORUS GeForce RTX 3090 Xtreme 24G Gra...,Gigabyte,AORUS RTX 3090 Xtreme 24G,"[gigabyte, aorus, rtx, 3090, xtreme, 24g]",24 GB,GDDR6X,319 mm,PCIe x16,GeForce RTX 3090,1395 MHz,1860 MHz,G-Sync,"$3,399.99"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,"VisionTek Radeon 5450 2GB DDR3 (DVI-I, HDMI, V...",VisionTek,Radeon 5450,"[visiontek, radeon, 5450]",2 GB,DDR3,NaN,PCIe x16,Radeon HD 5450,650 MHz,1000 MHz,None,$64.99
307,Diamond Multimedia AMD Radeon HD 5450 PCI Expr...,DiamondMultimedia,5450PE31G,"[diamondmultimedia, 5450pe31g]",1 GB,DDR3,152 mm,PCIe x16,Radeon HD 5450,650 MHz,1000 MHz,None,$54.99
308,Sapphire Technology Technology Radeon 11276-02...,SapphireTechnology,Radeon RX VEGA 56,"[sapphiretechnology, radeon, rx, vega, 56]",8 GB,HBM2,282 mm,PCIe x16,Radeon RX VEGA 56,1208 MHz,1512 MHz,FreeSync,NaN
309,ZOTAC Gaming GeForce RTX 2080 Ti AMP Extreme 1...,ZOTAC,GeForce RTX 2080 Ti AMP Extreme,"[zotac, geforce, rtx, 2080, ti, amp, extreme]",11 GB,GDDR6,324 mm,PCIe x16,GeForce RTX 2080 Ti,1545 MHz,1815 MHz,G-Sync,NaN


In [88]:
gpu_csv.drop_duplicates(subset=['name'],inplace= True)
gpu_csv

,name,brand,model,keywords,memory,memory_interface,length,interface,chipset,base_clock,clock_speed,frame_sync,amazon
0,MSI Gaming GeForce RTX 3090 24GB GDRR6X 384-Bi...,MSI,RTX 3090 GAMING X TRIO 24G,"[msi, rtx, 3090, gaming, x, trio, 24g]",24 GB,GDRR6X,335 mm,PCIe x16,GeForce RTX 3090,1395 MHz,1785 MHz,G-Sync,"$1,289.99"
1,"EVGA GeForce RTX 3090 FTW3 Ultra Gaming, 24GB ...",EVGA,RTX 3090 FTW3 Ultra,"[evga, rtx, 3090, ftw3, ultra]",24 GB,GDDR6X,300 mm,PCIe x16,GeForce RTX 3090,1395 MHz,1800 MHz,G-Sync,"$1,199.99"
2,ASUS ROG Strix NVIDIA GeForce RTX 3090 White O...,ASUS,ROG Strix RTX 3090 OC Edition,"[asus, rog, strix, rtx, 3090, oc, edition]",24 GB,GDDR6X,318.5 mm,PCIe x16,GeForce RTX 3090,1395 MHz,1890 MHz,G-Sync,"$1,794.99"
3,MSI Gaming GeForce RTX 3090 24GB GDRR6X 384-Bi...,MSI,RTX 3090 SUPRIM X 24G,"[msi, rtx, 3090, suprim, x, 24g]",24 GB,GDDR6X,336 mm,PCIe x16,GeForce RTX 3090,1395 MHz,1875 MHz,G-Sync,"$2,017.21"
4,GIGABYTE AORUS GeForce RTX 3090 Xtreme 24G Gra...,Gigabyte,AORUS RTX 3090 Xtreme 24G,"[gigabyte, aorus, rtx, 3090, xtreme, 24g]",24 GB,GDDR6X,319 mm,PCIe x16,GeForce RTX 3090,1395 MHz,1860 MHz,G-Sync,"$3,399.99"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,"VisionTek Radeon 5450 2GB DDR3 (DVI-I, HDMI, V...",VisionTek,Radeon 5450,"[visiontek, radeon, 5450]",2 GB,DDR3,NaN,PCIe x16,Radeon HD 5450,650 MHz,1000 MHz,None,$64.99
307,Diamond Multimedia AMD Radeon HD 5450 PCI Expr...,DiamondMultimedia,5450PE31G,"[diamondmultimedia, 5450pe31g]",1 GB,DDR3,152 mm,PCIe x16,Radeon HD 5450,650 MHz,1000 MHz,None,$54.99
308,Sapphire Technology Technology Radeon 11276-02...,SapphireTechnology,Radeon RX VEGA 56,"[sapphiretechnology, radeon, rx, vega, 56]",8 GB,HBM2,282 mm,PCIe x16,Radeon RX VEGA 56,1208 MHz,1512 MHz,FreeSync,NaN
309,ZOTAC Gaming GeForce RTX 2080 Ti AMP Extreme 1...,ZOTAC,GeForce RTX 2080 Ti AMP Extreme,"[zotac, geforce, rtx, 2080, ti, amp, extreme]",11 GB,GDDR6,324 mm,PCIe x16,GeForce RTX 2080 Ti,1545 MHz,1815 MHz,G-Sync,NaN


In [89]:
#calling microcenter function from microcenter.py to scrape the listings
microcenter = mc_scrape()
mc_df = pd.DataFrame(microcenter)
mc_df

,name,prices
0,ASUS NVIDIA GeForce RTX 3090 Ti TUF Gaming Ove...,"$1,449.99"
1,MSI AMD Radeon RX 6900 XT Gaming X Trio Triple...,$709.99
2,ASUS NVIDIA GeForce RTX 3090 Ti ROG Strix LC O...,"$1,499.99"
3,EVGA NVIDIA GeForce RTX 3070 FTW3 Ultra Gaming...,$719.99
4,MSI NVIDIA GeForce RTX 3080 Ventus 3X Plus LHR...,$789.99
...,...,...
163,Sapphire Technology AMD Radeon RX 6700 XT Nitr...,$615.59
164,MSI NVIDIA GeForce RTX 4090 Gaming Trio Triple...,"$1,649.99"
165,Sapphire Technology AMD Radeon RX 6900 XT Toxi...,$999.99
166,PNY NVIDIA RTX A4500 Single Fan 20 GB GDDR6 PC...,"$1,499.99"


In [90]:
#creating a list for filtered dataframe
gpu_mc_names = []
mc_names = []
match_scores = []
model_number = []
mc_price = []

In [91]:
#filtering through the listings to see which listings are the closest match according to the main GPU CSV
#looping through main GPU CSV df to match with other df's
for gpu_index, gpu_row in gpu_csv.iterrows():
    #removing extra fluff the main csv writes
    gpu_name = gpu_row['name'].lower().split('graphics card')[0]
    test = len(gpu_row['keywords'])
    count = 0
    #grabbing extra variables for the search
    brand = gpu_row['brand'].lower()
    model_num = [num for num in gpu_row['keywords'] if num.isdigit()]
    #loop through the microcenter df
    for mc_index, mc_row in mc_df.iterrows():
        mc_name = mc_row['name'].lower().split('graphics card')[0]
        #loop through the keywords for the search
        for keyword in gpu_row['keywords']:
            if keyword in mc_row['name']:
                count += 1
            if count == test and (brand in mc_name):
                #using the match score created by StubHub to find how relative the listings are
                match_score = fuzz.ratio(gpu_name, mc_name)
                #from the tests used any listing over a match score of 50 would give a similar/relative listing
                if match_score >= 50:
                    gpu_mc_names.append(gpu_row['name'])
                    mc_names.append(mc_name)
                    match_scores.append(match_score)
                    model_number.append(''.join(model_num))
                    mc_price.append(mc_row['prices'])
                count = 0
        else:
            next

In [92]:
#create the filtered dataframe
mc_filtered_df = pd.DataFrame({'gpu_name':gpu_mc_names, 'microcenter_name':mc_names,'microcenter_price':mc_price,
                               'Match Score':match_scores,'Model Number':model_number})
mc_filtered_df.head()

,gpu_name,microcenter_name,microcenter_price,Match Score,Model Number
0,"EVGA GeForce RTX 3090 FTW3 Ultra Gaming, 24GB ...",evga nvidia geforce rtx 3090 ti ftw3 ultra tri...,"$2,199.99",60,3090
1,ASUS TUF Gaming NVIDIA GeForce RTX 3090 OC Edi...,asus nvidia geforce rtx 3080 rog strix gaming ...,$799.99,50,3090
2,EVGA 24G-P5-3975-KR GeForce RTX 3090 XC3 Ultra...,evga nvidia geforce rtx 3090 ti ftw3 ultra tri...,"$2,199.99",54,3090
3,GIGABYTE AORUS GeForce RTX 3080 Ti Xtreme 10G ...,gigabyte geforce rtx 3070 vision overclocked t...,$779.99,56,3080
4,ASUS TUF Gaming NVIDIA GeForce RTX 3080 OC Edi...,asus nvidia geforce rtx 3080 ti tuf overclocke...,"$1,049.99",55,3080


In [93]:
#extra filtering to remove listings that are just completely wrong
for i,r in mc_filtered_df.iterrows():
    if r['Model Number'] not in r['microcenter_name']:
            mc_filtered_df.drop(i,inplace=True)
mc_filtered_df.drop_duplicates(subset='microcenter_name', keep='first',inplace=True)
mc_filtered_df.drop_duplicates(subset='gpu_name', keep='first',inplace=True)
mc_filtered_df.reset_index(inplace=True)
mc_filtered_df.drop(['Match Score','Model Number','index'],axis=1,inplace=True)

In [94]:
#double check
mc_filtered_df.head()

,gpu_name,microcenter_name,microcenter_price
0,"EVGA GeForce RTX 3090 FTW3 Ultra Gaming, 24GB ...",evga nvidia geforce rtx 3090 ti ftw3 ultra tri...,"$2,199.99"
1,ASUS TUF Gaming NVIDIA GeForce RTX 3080 OC Edi...,asus nvidia geforce rtx 3080 ti tuf overclocke...,"$1,049.99"
2,"EVGA GeForce RTX 3070 Ti FTW3 Ultra Gaming, 08...",evga nvidia geforce rtx 3070 xc3 ultra gaming ...,$659.99
3,ASUS ROG Strix NVIDIA GeForce RTX 3070 Ti OC E...,asus nvidia geforce rtx 3070 ti tuf gaming ove...,$699.99
4,ASUS Dual AMD Radeon RX 6700 XT Standard Editi...,asus amd radeon rx 6700 xt dual dual-fan 12gb ...,$729.99


In [18]:
#starting bestbuy search and scrape
bestbuy = bb_scrape()
bb_df = pd.DataFrame(bestbuy)
bb_df


,title,prices
0,XFX - SPEEDSTER SWFT210 AMD Radeon RX 6600 Cor...,$259.99
1,NVIDIA - GeForce RTX 3080 Ti 12GB GDDR6X PCI E...,$899.99
2,MSI - NVIDIA GeForce RTX 3060 Ventus 3X 12G OC...,$379.99
3,PNY - NVIDIA GeForce GTX 1650 4GB GDDR6 PCI Ex...,$179.99
4,PNY - NVIDIA GeForce RTX 3070 8GB GDDR6 PCI Ex...,$579.99
...,...,...
167,GIGABYTE - AMD Radeon RX 6600 XT GAMING OC 8GB...,$466.99
168,GIGABYTE - AMD Radeon RX 6800 GAMING OC 16GB G...,$712.99
169,GIGABYTE - AMD Radeon RX 6600 XT GAMING OC PRO...,$473.99
170,GIGABYTE - AMD Radeon RX 6800 AORUS MASTER 16G...,$820.99


In [31]:
bb_df.drop_duplicates(subset=['title'],inplace= True)
bb_df

,title,prices
0,XFX - SPEEDSTER SWFT210 AMD Radeon RX 6600 Cor...,$259.99
1,NVIDIA - GeForce RTX 3080 Ti 12GB GDDR6X PCI E...,$899.99
2,MSI - NVIDIA GeForce RTX 3060 Ventus 3X 12G OC...,$379.99
3,PNY - NVIDIA GeForce GTX 1650 4GB GDDR6 PCI Ex...,$179.99
4,PNY - NVIDIA GeForce RTX 3070 8GB GDDR6 PCI Ex...,$579.99
...,...,...
143,GIGABYTE - AMD Radeon RX 6600 XT GAMING OC 8GB...,$466.99
144,GIGABYTE - AMD Radeon RX 6800 GAMING OC 16GB G...,$712.99
145,GIGABYTE - AMD Radeon RX 6600 XT GAMING OC PRO...,$473.99
146,GIGABYTE - AMD Radeon RX 6800 AORUS MASTER 16G...,$820.99


In [72]:
#cleaning the data as some of the listings from best buy has the string 'new!'
for i,r in bb_df.iterrows():
    test = r['title'].lower().split('graphics card')[0]
    if 'new!' in test:
        r['title'] = test[4:]

In [73]:
#creating list of variables for the dataframe
gpu_bb_names = []
bb_names = []
bb_match_scores = []
bb_model_number = []
bb_price = []

In [74]:
#filtering through the listings to see which listings are the closest match according to the main GPU CSV
#looping through main GPU CSV df to match with other df's
for gpu_index, gpu_row in gpu_csv.iterrows():
    #removing extra fluff the main csv writes
    gpu_name = gpu_row['name'].lower().split('graphics card')[0]
    test = len(gpu_row['keywords'])
    count = 0
    #grabbing extra variables for the search
    brand = gpu_row['brand'].lower()
    model_num = [num for num in gpu_name.split() if num.isdigit()]
    #loop through the bestbuy df
    for bb_index, bb_row in bb_df.iterrows():
        bb_name = bb_row['title'].lower().split('graphics card')[0]
        #loop through the keywords for the search
        for keyword in gpu_row['keywords']:
            if keyword in bb_row['title']:
                count += 1
            if count == test and (brand in bb_name):
                #using the match score created by StubHub to find how relative the listings are
                match_score = fuzz.ratio(gpu_name, bb_name)
                if match_score >= 50:
                    #from the tests used, any listing over a match score of 50 would give a similar/relative listing
                    # print(bb_name)
                    gpu_bb_names.append(gpu_row['name'])
                    bb_names.append(bb_name)
                    bb_match_scores.append(match_score)
                    bb_model_number.append(''.join(model_num))
                    bb_price.append(bb_row['prices'])
                count = 0
        else:
            next

In [75]:
#creating the filtered bestbuy dataframe
bb_filtered_df = pd.DataFrame({'gpu_name':gpu_bb_names, 'bestbuy_name':bb_names,'bestbuy_price':bb_price,
                               'Match Score':bb_match_scores,'Model Number':bb_model_number})
bb_filtered_df.head()

,gpu_name,bestbuy_name,bestbuy_price,Match Score,Model Number
0,"EVGA GeForce RTX 3090 FTW3 Ultra Gaming, 24GB ...",evga - nvidia geforce rtx 3070 8gb xc3 ultra g...,$559.99,52,3090
1,GIGABYTE AORUS GeForce RTX 3090 Xtreme 24G Gra...,gigabyte - nvidia geforce rtx 3060 ti vision o...,$459.99,54,3090
2,GIGABYTE AORUS GeForce RTX 3090 Xtreme 24G Gra...,gigabyte - nvidia geforce rtx 4090 gaming oc 2...,"$1,699.99",57,3090
3,GIGABYTE AORUS GeForce RTX 3090 Xtreme 24G Gra...,gigabyte - nvidia geforce rtx 3090 gaming oc 2...,"$1,199.99",59,3090
4,GIGABYTE AORUS GeForce RTX 3090 Xtreme 24G Gra...,gigabyte - nvidia geforce gt 730 2gb pci expre...,$64.99,52,3090


In [76]:
for i,r in bb_filtered_df.iterrows():
    if r['Model Number'][:3] not in r['bestbuy_name']:
        bb_filtered_df.drop(i,inplace=True)
    elif r['Model Number'] == '':
        bb_filtered_df.drop(i,inplace=True)
bb_filtered_df.reset_index(inplace=True)        
bb_filtered_df.drop(['Match Score','Model Number','index'],axis=1,inplace=True)
bb_filtered_df.head()

,gpu_name,bestbuy_name,bestbuy_price
0,GIGABYTE AORUS GeForce RTX 3090 Xtreme 24G Gra...,gigabyte - nvidia geforce rtx 3090 gaming oc 2...,"$1,199.99"
1,Gigabyte GeForce RTX 3090 GAMING OC 24G Graphi...,gigabyte - nvidia geforce rtx 3090 aorus maste...,"$1,979.99"
2,Gigabyte GeForce RTX 3090 GAMING OC 24G Graphi...,gigabyte - nvidia geforce rtx 3090 eagle oc 24...,"$2,042.99"
3,PNY GeForce RTX 3080 10GB XLR8 Gaming Revel Ep...,pny - nvidia geforce rtx 3080 12gb gddr6x pci ...,$739.99
4,PNY GeForce RTX 3080 10GB XLR8 Gaming Revel Ep...,pny - nvidia geforce rtx 3080 10gb xlr8 gaming...,$729.99


In [77]:
bb_filtered_df.drop_duplicates(subset=['gpu_name'],inplace= True)
bb_filtered_df

,gpu_name,bestbuy_name,bestbuy_price
0,GIGABYTE AORUS GeForce RTX 3090 Xtreme 24G Gra...,gigabyte - nvidia geforce rtx 3090 gaming oc 2...,"$1,199.99"
1,Gigabyte GeForce RTX 3090 GAMING OC 24G Graphi...,gigabyte - nvidia geforce rtx 3090 aorus maste...,"$1,979.99"
3,PNY GeForce RTX 3080 10GB XLR8 Gaming Revel Ep...,pny - nvidia geforce rtx 3080 12gb gddr6x pci ...,$739.99
5,PNY GeForce RTX 3080 10GB XLR8 Gaming Uprising...,pny - nvidia geforce rtx 3080 12gb gddr6x pci ...,$739.99
7,NVIDIA GEFORCE RTX 2080 Ti Founders Edition,evga - nvidia geforce rtx 2080 super xc ultra ...,$607.99


# connect to local database

In [78]:
rds_connection_string = 'postgres:postgres@localhost:5432/project 2'
engine = create_engine(f'postgresql://{rds_connection_string}')

In [79]:
engine.table_names() #check for tables

C:\Users\nhle9\AppData\Local\Temp\ipykernel_40760\707408201.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names() #check for tables


['gpu_kaggle',
 'microcenter',
 'bestbuy',
 'microcenter_filtered',
 'bestbuy_filtered']

In [96]:
#Use pandas to load csv converted DataFrame into databse
gpu_csv.to_sql(name='gpu_kaggle', con=engine, if_exists='append', index=False)


302

In [97]:
mc_df.to_sql(name='microcenter', con=engine, if_exists='append', index=False)

168

In [98]:
bb_df.to_sql(name='bestbuy', con=engine, if_exists='append', index=False)

138

In [99]:
mc_filtered_df.to_sql(name='microcenter_filtered', con=engine, if_exists='append', index=False)

7

In [100]:
bb_filtered_df.to_sql(name='bestbuy_filtered', con=engine, if_exists='append', index=False)

5

In [65]:
#confirm that data has been added
pd.read_sql_query('select * from microcenter',con=engine).head()

,name,prices
0,ASUS NVIDIA GeForce RTX 3090 Ti TUF Gaming Ove...,"$1,449.99"
1,MSI AMD Radeon RX 6900 XT Gaming X Trio Triple...,$709.99
2,ASUS NVIDIA GeForce RTX 3090 Ti ROG Strix LC O...,"$1,499.99"
3,EVGA NVIDIA GeForce RTX 3070 FTW3 Ultra Gaming...,$719.99
4,MSI NVIDIA GeForce RTX 3080 Ventus 3X Plus LHR...,$789.99
